<a href="https://colab.research.google.com/github/jatyrAR/Computational_statistique/blob/master/DQN_project_MVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer 1__: 
The goal is to act accordingly to the best policy known by the agent. The problem is that, if the agent only act greedely with a few known policy, it will not explore the set of actions and therefore the actions took might be far from optimal. Here the idea is that with probability epsilon, we take a random action, otherwise we take the best known policy. This is a way to solve the exploration/exploitation dilema associated with those problems.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature = 0.3
epochs_train= 200 # set small when debugging
epochs_test= 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer 2__ :

Position enables one to store the previous positions of the agent and board is used to store what the agent have already seen, and eat in some cases. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return  np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
    
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)
            prev_state = state
            state, reward, game_over = env.act(action)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.5/11.0. Average score (-1.5)
Win/lose count 7.5/8.0. Average score (-1.0)
Win/lose count 13.5/14.0. Average score (-0.8333333333333334)
Win/lose count 9.5/14.0. Average score (-1.75)
Win/lose count 2.5/8.0. Average score (-2.5)
Win/lose count 7.5/13.0. Average score (-3.0)
Win/lose count 8.0/11.0. Average score (-3.0)
Win/lose count 8.0/12.0. Average score (-3.125)
Win/lose count 8.0/10.0. Average score (-3.0)
Win/lose count 5.0/9.0. Average score (-3.1)
Win/lose count 11.5/11.0. Average score (-2.772727272727273)
Win/lose count 14.5/16.0. Average score (-2.6666666666666665)
Win/lose count 12.0/14.0. Average score (-2.6153846153846154)
Win/lose count 7.5/12.0. Average score (-2.75)
Win/lose count 10.0/13.0. Average score (-2.7666666666666666)
Win/lose count 13.5/14.0. Average score (-2.625)
Win/lose count 10.5/14.0. Average score (-2.676470588235294)
Win/lose count 3.5/15.0. Average score (-3.1666666666666665)
Win/lose count 10.0/12.0. Average score (-3.105263157894736

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__: 

We have : \begin{equation*}
\begin{aligned}
    Q^\pi(s,a) &= \mathbb E_{p^\pi}\left[
        \sum_{t=0} \gamma^{t}r(s_t,a_t) | s_1=s, a_1=a
    \right]  \\
    &= r(s,a) + \gamma \mathbb E_{p^\pi}\left[
        \sum_{t=1} \gamma^{t-1}r(s_t,a_t) | s_1=s, a_1=a
    \right]  \\
    &= r(s,a) + \gamma \mathbb E_{(s',a')\sim p(\cdot|s,a)}\left[
        \mathbb E_{p^\pi}\left[
        \sum_{t=0} \gamma^{t}r(s_t,a_t) | s_0=s', a_0=a'
        \right]
    \right]  \\
    &= r(s,a) + \gamma \mathbb E_{(s',a')\sim p(\cdot|s,a)}\left[
        Q^\pi(s',a')
    \right]
\end{aligned}
\end{equation*}

Therefore, 
$$
\begin{aligned}
    Q^*(s,a) &= Q^{\pi^*}(s,a)  \\
    &= r(s,a) + \gamma \mathbb E_{(s',a')\sim p^{\pi^*}(s,a)}[
        Q^{\pi^*}(s',a')
    ]
\end{aligned}
$$
since $ \pi^* $ is deterministic ( $argmax(Q^*)$) :
$$
\begin{aligned}
    Q^*(s,a) = r(s,a) + \gamma \mathbb E_{s'\sim p(\cdot|s,a)}
    \left[
        \max_{a'} Q^*(s',a')
    \right]
\end{aligned}
$$

Since $Q^*$ should satisfy the previous equation, we can choose to minimise the $L^2$ distance $\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) <= self.max_memory:
            self.memory.append(m)
        else:
            del self.memory[0]  

    def random_access(self, size=1):
        idx = np.random.randint(0,len(self.memory),size=size) 
        return np.array(self.memory)[idx]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        next_states = np.zeros((self.batch_size, 5,5,self.n_state))
        
        samples = self.memory.random_access(self.batch_size) # draw batch_size samples from memory
        actions = []
        rewards = []
        dones = []
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = samples[i] 
            # store corresponding  s,a,s',r, game_over in arrays for later use in Batch mode
            input_states[i] = s_
            next_states[i] = n_s_
            actions.append(a_)
            rewards.append(r_)
            dones.append(game_over_)
        
        # Batch-prediction to make computations faster    
        target_q = self.model.predict(input_states)
        next_q = self.model.predict(next_states)
        
        for i in range(self.batch_size):
            if dones[i]:
                # If the state is terminal, it's Q value is simply the collected reward
                target_q[i, actions[i]] = rewards[i] 
            else:
                # target according to equation proved in question 5
                target_q[i, actions[i]] = rewards[i] + self.discount * next_q[i].max()
            

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model 
        in_shape = (5, 5, self.n_state)
        
        fc1_size = 32 # size of first hidden layer
        fc2_size = 32 # size of second hidden layer
        
        out_shape = self.n_action
        
        # Input: 5x5 neighborhood of the current state s.
        
        model: Sequential = Sequential([
            keras.layers.Flatten(input_shape=in_shape),
            Dense(fc1_size, activation='relu'),
            Dense(fc2_size, activation='relu'),
            Dense(out_shape)  # output layer
        ])
        # Output: Prediction of the Q values Q(s, a) for all actions a, s being the current state.
        
        model.compile(keras.optimizers.adam(lr=lr), "mse")
        self.model = model
        

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-4, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 200, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/200 | Loss 0.0105 | Win/lose count 2.5/6.0 (-3.5)
Epoch 001/200 | Loss 0.0160 | Win/lose count 4.5/7.0 (-2.5)
Epoch 002/200 | Loss 0.0168 | Win/lose count 4.0/4.0 (0.0)
Epoch 003/200 | Loss 0.0146 | Win/lose count 2.0/4.0 (-2.0)
Epoch 004/200 | Loss 0.0136 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/200 | Loss 0.0082 | Win/lose count 2.5/3.0 (-0.5)
Epoch 006/200 | Loss 0.0040 | Win/lose count 3.0/6.0 (-3.0)
Epoch 007/200 | Loss 0.0022 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/200 | Loss 0.0075 | Win/lose count 5.0/5.0 (0.0)
Epoch 009/200 | Loss 0.0139 | Win/lose count 7.5/5.0 (2.5)
Epoch 010/200 | Loss 0.0030 | Win/lose count 2.0/1.0 (1.0)
Epoch 011/200 | Loss 0.0353 | Win/lose count 7.5/7.0 (0.5)
Epoch 012/200 | Loss 0.0569 | Win/lose count 6.0/3.0 (3.0)
Epoch 013/200 | Loss 0.0366 | Win/lose count 1.5/3.0 (-1.5)
Epoch 014/200 | Loss 0.0095 | Win/lose count 3.

Epoch 130/200 | Loss 0.0023 | Win/lose count 10.0/1.0 (9.0)
Epoch 131/200 | Loss 0.0606 | Win/lose count 8.5/2.0 (6.5)
Epoch 132/200 | Loss 0.0485 | Win/lose count 2.5/2.0 (0.5)
Epoch 133/200 | Loss 0.0033 | Win/lose count 3.0/2.0 (1.0)
Epoch 134/200 | Loss 0.0039 | Win/lose count 6.5/1.0 (5.5)
Epoch 135/200 | Loss 0.0054 | Win/lose count 11.5/3.0 (8.5)
Epoch 136/200 | Loss 0.0018 | Win/lose count 10.0/2.0 (8.0)
Epoch 137/200 | Loss 0.0786 | Win/lose count 5.0/1.0 (4.0)
Epoch 138/200 | Loss 0.0031 | Win/lose count 9.5/1.0 (8.5)
Epoch 139/200 | Loss 0.0044 | Win/lose count 9.5/2.0 (7.5)
Epoch 140/200 | Loss 0.0008 | Win/lose count 9.0/2.0 (7.0)
Epoch 141/200 | Loss 0.0057 | Win/lose count 4.5/2.0 (2.5)
Epoch 142/200 | Loss 0.0033 | Win/lose count 6.0/2.0 (4.0)
Epoch 143/200 | Loss 0.0049 | Win/lose count 11.0/5.0 (6.0)
Epoch 144/200 | Loss 0.0060 | Win/lose count 3.0/0 (3.0)
Epoch 145/200 | Loss 0.0037 | Win/lose count 14.5/2.0 (12.5)
Epoch 146/200 | Loss 0.0386 | Win/lose count 18.0/3.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        in_shape = (5, 5, self.n_state)
        conv1_shape = 32
        conv2_shape = 64
        conv3_shape = 32
        out_shape = self.n_action
        
        model = Sequential([
            Conv2D(conv1_shape, 3, input_shape=in_shape, activation='relu', padding="same"),
            Conv2D(conv2_shape, 3, activation='relu', padding="same"),
            Conv2D(conv3_shape, 3, activation='relu', padding="same"),
            keras.layers.Flatten(),
            Dense(out_shape)  
        ])
        
        
        model.compile(keras.optimizers.adam(lr=lr), "mse")
        
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-4, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,200,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/200 | Loss 0.0078 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/200 | Loss 0.0060 | Win/lose count 3.5/1.0 (2.5)
Epoch 002/200 | Loss 0.0034 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/200 | Loss 0.0019 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/200 | Loss 0.0015 | Win/lose count 6.5/7.0 (-0.5)
Epoch 005/200 | Loss 0.0704 | Win/lose count 7.5/3.0 (4.5)
Epoch 006/200 | Loss 0.0017 | Win/lose count 7.5/0 (7.5)
Epoch 007/200 | Loss 0.0018 | Win/lose count 3.5/3.0 (0.5)
Epoch 008/200 | Loss 0.0023 | Win/lose count 9.5/4.0 (5.5)
Epoch 009/200 | Loss 0.0021 | Win/lose count 2.5/1.0 (1.5)
Epoch 010/200 | Loss 0.0502 | Win/lose count 3.5/3.0 (0.5)
Epoch 011/200 | Loss 0.0012 | Win/lose count 10.5/1.0 (9.5)
Epoch 012/200 | Loss 0.0014 | Win/lose count 2.5/2.0 (0.5)
Epoch 013/200 | Loss 0.0029 | Win/lose count 9.0/3.0 (6.0)
Epoch 014/200 | Loss 0.0018 | Win/lose count 5.0/3.0 (2.0)
Epoch 015/200 | Loss 0.0036 | Win/lose count 6.5/4.0 (2.5)
Epoch 016/200 | Loss 0.0062 | Win/lose count 10.0/4.0

Epoch 138/200 | Loss 0.0008 | Win/lose count 10.0/4.0 (6.0)
Epoch 139/200 | Loss 0.0004 | Win/lose count 12.5/5.0 (7.5)
Epoch 140/200 | Loss 0.0017 | Win/lose count 8.0/3.0 (5.0)
Epoch 141/200 | Loss 0.0009 | Win/lose count 7.0/3.0 (4.0)
Epoch 142/200 | Loss 0.0339 | Win/lose count 9.0/2.0 (7.0)
Epoch 143/200 | Loss 0.0012 | Win/lose count 10.5/5.0 (5.5)
Epoch 144/200 | Loss 0.0014 | Win/lose count 15.0/0 (15.0)
Epoch 145/200 | Loss 0.0017 | Win/lose count 8.5/4.0 (4.5)
Epoch 146/200 | Loss 0.0753 | Win/lose count 10.0/1.0 (9.0)
Epoch 147/200 | Loss 0.0024 | Win/lose count 8.0/2.0 (6.0)
Epoch 148/200 | Loss 0.0095 | Win/lose count 4.5/1.0 (3.5)
Epoch 149/200 | Loss 0.0032 | Win/lose count 14.0/2.0 (12.0)
Epoch 150/200 | Loss 0.0697 | Win/lose count 10.0/3.0 (7.0)
Epoch 151/200 | Loss 0.0010 | Win/lose count 6.0/2.0 (4.0)
Epoch 152/200 | Loss 0.0022 | Win/lose count 7.0/3.0 (4.0)
Epoch 153/200 | Loss 0.0023 | Win/lose count 6.5/0 (6.5)
Epoch 154/200 | Loss 0.0014 | Win/lose count 0.5/1.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=1e-4, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=1e-4, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 3.5/0. Average score (2.25)
Win/lose count 3.5/0. Average score (2.6666666666666665)
Win/lose count 3.0/0. Average score (2.75)
Win/lose count 1.0/0. Average score (2.4)
Win/lose count 1.0/0. Average score (2.1666666666666665)
Win/lose count 1.5/0. Average score (2.0714285714285716)
Win/lose count 7.0/0. Average score (2.6875)
Win/lose count 2.5/0. Average score (2.6666666666666665)
Win/lose count 1.5/0. Average score (2.55)
Win/lose count 0/0. Average score (2.3181818181818183)
Win/lose count 1.0/0. Average score (2.2083333333333335)
Win/lose count 9.5/0. Average score (2.769230769230769)
Win/lose count 3.0/1.0. Average score (2.7142857142857144)
Win/lose count 4.0/0. Average score (2.8)
Win/lose count 1.5/0. Average score (2.71875)
Win/lose count 1.5/0. Average score (2.6470588235294117)
Win/lose count 4.5/0. Average score (2.75)
Win/lose count 2.0/0. Average score (2.710526315789474)
Win/lose count 5.0/0. Avera

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

__Answer 9__: 

Both algorithms are not performing well. The CNN gives slightly better results but the problem is that the exploration of the environement is not suficent, therefore the agent gets stuck in area that it has already visited. Increasing the temperature helps a little but it is far from enough.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Total no. of games won
    score = 0
    loss = 0
    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            action = agent.act(state, train=True)
            prev_state = state
            state, reward, game_over = env.act(action,train=True)
            
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
            
            
            
            # with the original rewards from "normal" environment
            # Update the counters
            original_reward = reward + env.penalty # recover original reward by substracting the last penalty
            if original_reward > 0:
                win = win + original_reward
            else :
                lose = lose - original_reward
        
        # Updating the epsilon of the agent (Decreasing ϵ -greedy exploration)
        new_epsilon = 0.9*agent.epsilon
        agent.set_epsilon(new_epsilon)
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

#We make the new class a subclass of Environment to inherit useful methods       
class EnvironmentExploring(Environment):
    
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((self.grid_size, self.grid_size)) # penalties for already visited cells (to encourage exploration)
        self.penalty = 0  # needed to keep track of the original reward for testing purposes
        
    def act(self, action, train=False):

        # get reward and game_over from "normal" environment.
        state, reward, game_over = super(EnvironmentExploring, self).act(action)
    
        if train:
            # penalize the current state if already visited to force exploring new states
            reward = reward - self.malus_position[self.x, self.y]
        self.penalty = self.malus_position[self.x, self.y]  # needed to keep track of the original reward for testing purposes
        self.malus_position[self.x, self.y] = 0.2
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over
        
    def reset(self):
        
        state = super(EnvironmentExploring, self).reset()  # call parent reset method, drop unaugmented state
    
        self.malus_position = np.zeros((self.grid_size, self.grid_size))  # No penalties since we haven't visited any state yet.
        self.penalty = 0
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state
    



In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-4, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 50, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0112 | Win/lose count 9.0/15.0 (-6.0)
Epoch 001/050 | Loss 0.0093 | Win/lose count 10.5/7.0 (3.5)
Epoch 002/050 | Loss 0.0204 | Win/lose count 10.0/7.0 (3.0)
Epoch 003/050 | Loss 0.0089 | Win/lose count 18.0/5.0 (13.0)
Epoch 004/050 | Loss 0.0329 | Win/lose count 19.5/4.0 (15.5)
Epoch 005/050 | Loss 0.0123 | Win/lose count 8.5/3.0 (5.5)
Epoch 006/050 | Loss 0.0196 | Win/lose count 20.5/0.0 (20.5)
Epoch 007/050 | Loss 0.0111 | Win/lose count 17.0/3.0 (14.0)
Epoch 008/050 | Loss 0.0080 | Win/lose count 22.5/2.0 (20.5)
Epoch 009/050 | Loss 0.0550 | Win/lose count 25.5/3.0 (22.5)
Epoch 010/050 | Loss 0.0067 | Win/lose count 17.5/4.0 (13.5)
Epoch 011/050 | Loss 0.0088 | Win/lose count 20.5/3.0 (17.5)
Epoch 012/050 | Loss 0.0072 | Win/lose count 19.5/1.0 (18.5)
Epoch 013/050 | Loss 0.0055 | Win/lose count 20.0/2.0 (18.0)
Epoch 014/050 | Loss 0.0043 | Win/lose count 23.0/3.0 (20.0)
Epoch 015/050 | Loss 0.0059 | Win/lose count 15.5/2.0 (13.5)
Epoch 016/050 | Loss 0.0060 

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 14.5/2.0. Average score (12.5)
Win/lose count 3.5/0. Average score (8.0)
Win/lose count 8.5/1.0. Average score (7.833333333333333)
Win/lose count 5.0/0. Average score (7.125)
Win/lose count 5.5/1.0. Average score (6.6)
Win/lose count 4.5/0. Average score (6.25)
Win/lose count 6.0/3.0. Average score (5.785714285714286)
Win/lose count 11.5/0. Average score (6.5)
Win/lose count 5.5/0. Average score (6.388888888888889)
Win/lose count 12.5/2.0. Average score (6.8)
Win/lose count 9.0/1.0. Average score (6.909090909090909)
Win/lose count 2.0/0. Average score (6.5)
Win/lose count 12.5/1.0. Average score (6.884615384615385)
Win/lose count 6.0/0. Average score (6.821428571428571)
Win/lose count 9.0/1.0. Average score (6.9)
Win/lose count 6.5/0. Average score (6.875)
Win/lose count 10.0/1.0. Average score (7.0)
Win/lose count 16.0/1.0. Average score (7.444444444444445)
Win/lose count 7.0/0. Average score (7.421052631578948)
Win/lose count 8.0/1.0. Average score (7.4)
Final score: 7

It works :)

***